# SQL for AI Projects

## Natural Language Processing Challenge

In this Jupyter notebook - we'll quickly setup the DuckDB database, get you familiar with this Google Colab setup and then we'll dive into the NLP challenge exercises for the SQL for AI Projects course!

### Challenge Exercises

1. Explore and transform tourist location site `html_data` column from the `locations` table
2. Apply NLP analysis techniques using the `reviews` tables
3. Analyze experiment results by joining multiple tables using A/B test framework

### Database Setup

First things first, let's load up our Python libraries and setup access to our database.

Don't worry if you're not familiar with Python - we'll just need to run the very first cell to initialize our SQL instance and there will be clear instructions whenever there is some non-SQL components.

### Getting Started

To execute each cell in this notebook - you can click on the play button on the left of each cell or you could simply hit the `Run all` button on the top of the notebook just below the menu toolbar.

This cell below will help us download and connect to a DuckDB database object within this notebook's temporary environment.

There will also be a few outputs in the same cell including the following:

* An interactive entity relationship diagram for our database is also as an output from the following cell. This will help us visualize all of the database tables and their relevant primary and foreign keys.

In [ ]:
# Initial setup steps
# ====================

# These pip install commands are required for Google Collab notebook environment
!pip install --upgrade --quiet duckdb==1.3.1
!pip install --quiet duckdb-engine==0.17.0
!pip install --quiet jupysql==0.11.1

# Also need to setup Git LFS for large file dowloads
!apt-get install git-lfs -y
!git lfs install

# Clone GitHub repo into a "data" folder
!git clone https://github.com/LinkedInLearning/real-world-data-and-AI-challenges-with-SQL-3813163.git data

# Need to change directory into "data" to run download database object
%cd data
!git lfs pull

# Then we need to change directory back up so all our paths are correct!
%cd ..

# Time to import all our Python packages
import duckdb
import textwrap
import pandas as pd
from pathlib import Path
from IPython.display import HTML, display

# Load the jupysql extension to enable us to run SQL code in code cells
%load_ext sql

# We can now set some basic Pandas settings for rendering SQL outputs
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

# This is a convenience function to print long strings into multiple lines
def wrap_print(text):
    print(textwrap.fill(text, width=80))

# This is some boilerplate code to help us format printed output with wrapping
HTML("""
<style>
.output pre {
    white-space: pre-wrap;
    word-break: break-word;
}
</style>
""")

# Connecting to DuckDB
# ====================

# Setup the SQL connection
connection = duckdb.connect("data/data.db")
%sql connection

# Run a few test queries using both connections
tables = connection.execute("SHOW TABLES").fetchall()
table_names = [table[0] for table in tables]

preview_counts_list = []
for table_name in table_names:
    try:
        preview_counts_list.append(
            connection.execute(f"""
                SELECT '{table_name}' AS table_name,
                    COUNT(*) AS record_count
                FROM {table_name}""").fetchdf()
        )
    except Exception as e:
        print(f"❌ Could not preview table {table_name}: {e}")
        

print("✅ Database is now ready!")

print("\n📋 Show count of rows from each table in the database:")

# Combine all dataframes in preview_df_list
preview_counts_df = pd.concat(preview_counts_list, ignore_index=True)

display(preview_counts_df)

display(HTML('''
<iframe width="100%" height="600" src='https://dbdiagram.io/e/685279b3f039ec6d36c0c7e9/68527d19f039ec6d36c1813e'> </iframe>
'''
))

# 1. How to Run SQL Queries

Let's quickly see how we can run SQL code in our Jupyter notebook.

In our Colab environment we can run single or multi-line queries. We can also easily save the output of SQL queries as a local Pandas DataFrame object and even run subsequent SQL queries which can interact with these same DataFrame objects.

## 1.1 Single Line SQL Query

We can use our notebook magic `%sql` at the start of a notebook cell to run a single line of SQL to query our database.

Let's take a look at the first 5 rows from the `locations` table:

In [ ]:
%sql SELECT * FROM locations LIMIT 5;

## 1.2 Multi-Line SQL Query

We can also run multi-line SQL queries by using a different notebook magic `%%sql` where we now have 2 percentage signs.

We'll apply a filter on our `location` dataset and return 2 columns.

In [ ]:
%%sql
SELECT
  location_name,
  description
FROM locations
WHERE location_id = 1;

## 1.3 Saving SQL Outputs

By using the `<<` operator, we can assign the result of a SQL query (returned as a Pandas DataFrame) to a named Python variable in the notebook’s scope.

### 1.3.1 Single Line Assignment

We can specify the name of the output variable directly after the `%sql` or `%%sql` magic command.

In [ ]:
%sql single_magic_df << SELECT * FROM locations LIMIT 5;

We can now reference the Python variable directly as a Pandas DataFrame

In [ ]:
# Python notebook scope
single_magic_df

We can also use this same variable as a table reference within a DuckDB `SELECT` statement.

In [ ]:
%sql SELECT * FROM single_magic_df;

### 1.3.2 Multi-line Assignment

This assignment using `<<` also works with the `%%sql` (multi-line) magic command.

In [ ]:
%%sql multi_magic_df <<
SELECT
  location_name,
  description
FROM locations
WHERE location_id = 1;

In [ ]:
multi_magic_df

When referencing the Python variable within DuckDB, we can also use it inside a multi-line SQL query using the `%%sql` magic command.

In [ ]:
%%sql
SELECT *
FROM multi_magic_df;

# 2. Challenge Exercise #1

In this exercise, we’ll clean and prepare the html_data column from the locations table so it’s ready for NLP.

## 2.1 Inspect Raw Data

Let’s start by looking at a single row — specifically for Yosemite National Park — to see what kind of cleaning is needed.

We’ll use the .loc method in Pandas to inspect the raw HTML. In this case, our expression:

```python
yosemite_html_example_df.loc[0, "html_data"]
```

Means: “Get the value from the first row of the DataFrame, specifically from the html_data column.”

In [ ]:
%sql yosemite_html_example_df << SELECT html_data FROM locations WHERE location_id = 1;

# We'll save this variable for use in a later cell!
yosemite_raw_html_string = yosemite_html_example_df.loc[0, "html_data"]

print(yosemite_raw_html_string)